# Методы доступа к атрибутам

https://github.com/alexopryshko/advancedpython/tree/master/1

В предыдущей теме были рассмотрены дескрипторы. Они позволяют переопределять доступ к атрибутам класса изнутри атрибута. Тем не менее в питоне есть еще группа магических методов, которые вызываются при доступе к атрибутам со стороны объекта вызывающего класса:

- `__getattribute__(self, name)` - будет вызван при попытке получить значение атрибута. Если этот метод переопределён, стандартный механизм поиска значения атрибута не будет задействован. По умолчанию как раз он и лезет в `__dict__` объекта и вызывает в случае неудачи `__getattr__`:
- `__getattr__(self, name)` - будет вызван в случае, если запрашиваемый атрибут не найден обычным механизмом (в `__dict__` экземпляра, класса и т.д.)
- `__setattr__(self, name, value)` - будет вызван при попытке установить значение атрибута экземпляра. Если его переопределить, стандартный механизм установки значения не будет задействован.
- `__delattr__(self, name)` - используется при удалении атрибута.

В следующем примере показано, что `__getattr__` вызывается только тогда, когда стандартными средствами (заглянув в `__dict__` объекта и класса) найти атрибут не получается. При этом в нашем случае метод срабатывает для любых значений, не вызывая AttributeError

In [1]:
class A:
    def __getattr__(self, attr):
        print('__getattr__')
        return 42

    field = 'field'


a = A()
a.name = 'name'

print(a.__dict__, A.__dict__, end='\n\n\n')
print('a.name', a.name, end='\n\n')
print('a.field', a.field, end='\n\n')
print('a.random', a.random, end='\n\n')

{'name': 'name'} {'__module__': '__main__', '__getattr__': <function A.__getattr__ at 0x000001C291DEC9D8>, 'field': 'field', '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}


a.name name

a.field field

__getattr__
a.random 42



In [2]:
a.asdlubaslifubasfuib

__getattr__


42

А если переопределим `__getattribute__`, то даже на `__dict__` посмотреть не сможем.

In [3]:
class A:
    def __getattribute__(self, item):
        if item in self.__dict__:
            return self.__dict__[item]
        if item in self.__class__.__dict__:
            return self.__class__.__dict__[item]
        if item in object.__dict__:
            return object.__dict__[item]
        if '__getattr__' in self.__class__.__dict__:
            return self.__getattr__(self, item)
        raise AttributeError
        
        
        print('__getattribute__')
        return 42

    def __len__(self):
        return 0

    def test(self):
        print('test', self)

    field = 'field'


a = A()
a.name = 'name'

print('__dict__', getattr(a, "__dict__"), end='\n\n')
print('a.name', a.name, end='\n\n')
print('a.field', a.field, end='\n\n')
print('a.random', a.random, end='\n\n')
print('a.__len__', a.__len__, end='\n\n')
print('len(a)', len(a), end='\n\n')
print('type(a)...', type(a).__dict__['test'](a), end='\n\n')
print('A.field', A.field, end='\n\n')


__getattribute__
__dict__ 42

__getattribute__
a.name 42

__getattribute__
a.field 42

__getattribute__
a.random 42

__getattribute__
a.__len__ 42

len(a) 0

test <__main__.A object at 0x000001C291E6FA58>
type(a)... None

A.field field



In [5]:
a.test()

__getattribute__


TypeError: 'int' object is not callable

Переопределяя `__setattr__`, рискуем не увидеть наши добавляемые атрибуты объекта в `__dict__`

In [12]:
class A:
    def __setattr__(self, key, value):
        print('__setattr__')

    field = 'field'


a = A()
a.field = 1
a.a = 1
print('a.__dict__', a.__dict__, end='\n\n')
A.field = 'new'
print('A.field', A.field, end='\n\n')

__setattr__
__setattr__
a.__dict__ {}

A.field new



In [14]:
A.__dict__

mappingproxy({'__module__': '__main__',
              '__setattr__': <function __main__.A.__setattr__(self, key, value)>,
              'field': 'new',
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [13]:
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'field']

In [9]:
a.a

1

А таким образом можем разрешить нашему объекту возвращать только те атрибуты, название которых начинается на слово test. Теоретически, используя этот прием, можно реализовать истинно приватные атрибуты, но зачем?

In [15]:
class A:
    def __getattribute__(self, item):
        if 'test' in item or '__dict__' == item:
            return super().__getattribute__(item)
        else:
            raise AttributeError


a = A()
a.test_name = 1
a.name = 1
print('a.__dict__', a.__dict__)
print('a.test_name', a.test_name)
print('a.name', a.name)

a.__dict__ {'test_name': 1, 'name': 1}
a.test_name 1


AttributeError: 

In [ ]:
class A:
    def __init__(self):
        self.obj_field = 4
    
    class_field = 5
    
    data_descr = DataDescriptor()
    nondata_descr = NonDataDescriptor()
    
a = A()

a.liunyiuynliun

## Общий алгоритм получения атрибута

Чтобы получить значение атрибута attrname:
- Если определён метод `a.__class__.__getattribute__()`, то вызывается он и возвращается полученное значение.
- Если attrname это специальный (определённый python-ом) атрибут, такой как `__class__` или `__doc__`, возвращается его значение.
- Проверяется `a.__class__.__dict__` на наличие записи с attrname. Если она существует и значением является data дескриптор, возвращается результат вызова метода `__get__()` дескриптора. Также проверяются все базовые классы.
- Если в `a.__dict__` существует запись с именем attrname, возвращается значение этой записи.
- Проверяется `a.__class__.__dict__`, если в нём существует запись с attrname и это non-data дескриптор, возвращается результат `__get__()` дескриптора, если запись существует и там не дескриптор, возвращается значение записи. Также обыскиваются базовые классы.
- Если существует метод `a.__class__.__getattr__()`, он вызывается и возвращается его результат. Если такого метода нет — выкидывается `AttributeError`.

## Общий алгоритм назначения атрибута

Чтобы установить значение value атрибута attrname экземпляра a:
- Если существует метод `a.__class__.__setattr__()`, он вызывается.
- Проверяется `a.__class__.__dict__`, если в нём есть запись с attrname и это дескриптор данных — вызывается метод `__set__()` дескриптора. Также проверяются базовые классы.
- `a.__dict__` добавляется запись value с ключом attrname.

## Задание

Библиотека pandas предназначена для работы с табличными данными. В ней есть сущности DataFrame (по сути, сама таблица) и Series (колонка либо строка таблицы). У колонок внутри таблицы есть названия, притом получить колонку можно двумя способами:

- `dataframe.colname`
- `dataframe['colname']`

Задание: реализовать структуру данных ключ-значение, где и присваивание, и получение элементов можно будет производить обоими этими способами.